## Povprečne mesečne plače po občinah

### Projek pri podatkovnem rudarjenju

Podatkovno rudarjenje, projekt, `2. 3. 2020`  
**`Domen Ramšak`** **`Anže Grintal Ugovšek`**

Izbran problem (oziroma zbirka podatkov) je Povprečne mesečne plače po občinah, Slovenija, letno.

Problem zajema povprečne bruto in neto plače po občinah v Sloveniji, z dodatnimi podatki kot so opravljene (oz. plačane) nadure, število oseb, ki so opravljale nadure na mesec, število oseb, ki so prejele zaostala izplačila itd.

### Podatki

